In [199]:
TEST_DIR = "test/"
import csv
with open(TEST_DIR+'test.csv', 'r') as f:
    data = [i.split(",") for i in f.read().split()]
    listInput = []
    listTarget = []
    temp =[]
    temp2 = []
    
    def slice_per(source, step):
        return [source[i::step] for i in range(step)]

    for i in range(len(data)):
        for j in range(len(data[i])):
            if i!=0:
                if j == 0 or j == 1:
                    temp.append(data[i][j])
            # temp2 = (temp, 2)
        listInput = slice_per(temp, len(data)-1)
        
    for i in range(len(data)):
        for j in range(len(data[i])):
            if i!=0:
                if j == 2:
                    temp2.append(data[i][j])
            # temp2 = (temp, 2)
        listTarget = slice_per(temp2, len(data)-1)
        
print (listInput)
print (listTarget)

[['0', '1'], ['0', '0'], ['0', '1'], ['1', '1']]
[['0'], ['1'], ['1'], ['0']]


In [196]:
import pandas as pd
from components.Activation import Activation
from components.Layer import Layer

class FFNNModel:
    def __init__(self, layers):
        self.layers : list[Layer] = layers
        pass

    def setInitialInput(self, input: list[int]):
        if (len(self.layers) >= 1):
            self.layers[0].values = input

    def solve(self):
        for i in range(len(self.layers)-1):
            print("LAYER ==> ", i)
            idxLayerInput = i
            idxLayerTarget = i+1
            targetValues = []
            print("Input layer \t:", self.layers[idxLayerInput].values)
            tempSigma = []
            for nodeTarget in range(self.layers[idxLayerTarget].numOfNode):
                sigma = self.getSigma(idxLayerTarget, self.layers[idxLayerInput].values, nodeTarget)
                tempVal = Activation.active(sigma, self.layers[idxLayerTarget].activation)
                targetValues.append(tempVal)
                targetValues.append(tempVal)
                tempSigma.append(sigma)

            print("Sigma   :", tempSigma)
            self.layers[idxLayerTarget].values = targetValues
            # print(targetValues)
        return self.layers[len(self.layers)-1].values

    
    def getSigma(self, idxLayer, input, node):
        temp = 0
        for idx, w in enumerate(self.layers[idxLayer].weights[node]):
            if (idx != 0):
                temp += w * input[idx-1]
            else:
                temp += w
        return temp

    @staticmethod
    def getModelFromFile (path):
        MODELS_DIR = "models/"
        layers: list[Layer] = []
        df = pd.read_json(MODELS_DIR + path)	
        for layer in df.model.layers :
            layer_ = Layer(layer['activation'], layer['numOfNode'], layer['weights'], layer['values'])
            layers.append(layer_)

        return FFNNModel(layers)

In [197]:
fnn = FFNNModel.getModelFromFile('model.json')

def isCorrect(result, target):
    for idx, res in enumerate(result):
        if (int(res) != int(target[idx])):
            return False

    return True

def solveModel(inp, target):
    result = []
    count = 0
    for idx, test in enumerate(inp):
        fnn.setInitialInput(test)
        res = fnn.solve()(test, end=" -> ")
        print(res, end="  ")
        if (isCorrect(res, target[idx])):
            print("O")
            count += 1
        else:
            print("X")
        result.append(res)
        
    print("Akurasi: ", float(count)/len(inp))

def readInputFromFile(filename):

    return [[0, 0], [1,0], [0,1], [1,1]], [[0], [1], [1], [0]]
    
listInput = [[0, 0], [1,0], [0,1], [1,1]]
listTarget = [[0], [1], [1], [0]]

listInput, listTarget = readInputFromFile(TEST_DIR + "test.csv")

solveModel(listInput, listTarget)

LAYER ==>  0
Input layer 	: [0, 0]


AttributeError: 'int' object has no attribute 'values'

# Visualization

In [105]:
# Model Visualization

from graphviz import Digraph
f = Digraph('Feed Forward Neural Network', filename='model 1.gv')
f.attr('node', shape='circle', fixedsize='true', width='0.9')

for i in range(len(fnn.layers)) :
    if i != 0:
        if i == 1:
            for j in range(len(fnn.layers[i].weights)) :
                for k in range(len(fnn.layers[i].weights[j])) :
                    if k == 0:
                        if j == 0:
                            f.edge(f'bx', f'h{i}_{k}', str(
                            fnn.layers[i].weights[j][k]))
                        else:
                            f.edge(f'bx', f'h{i}_{k+1}', str(
                                fnn.layers[i].weights[j][k]))
                    else :
                        if j == 0:
                            if k == 2:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j+1][k]))
                            else:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j][k]))
                        else:
                            if k == 1:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j-1][k]))
                            else:
                                f.edge(f'x{j}', f'h{i}_{k-1}', str(
                                fnn.layers[i].weights[j][k]))

        elif i ==2 :
            for j in range(len(fnn.layers[i].weights)):
                for k in range(len(fnn.layers[i].weights[j])):
                    if k == 0:
                        f.edge(f'bhx{i-1}', f'h{i}_{j}', str(
                            fnn.layers[i].weights[j][k]))
                    elif k == 1 :
                        f.edge(f'h{i-1}_{j}', f'h{i}_{0}', str(
                            fnn.layers[i].weights[j][k]))
                    else:
                        f.edge(f'h{i-1}_{j+1}', f'h{i}_{0}', str(
                            fnn.layers[i].weights[j][k]))
       
f.view()

ModuleNotFoundError: No module named 'graphviz'